In [161]:
import pandas as pd

In [162]:
df1 = pd.read_csv("1992_CO_NBI.csv")

C:\Users\tobby\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,36,37,38,39,41,44,61,63,66,67,68,69,70,76,77,79,80,86,87,88,98,106,107,108,109,113,114,121,130,131,133) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [163]:
df2 = pd.read_csv("2019_CO_NBI.csv")

C:\Users\tobby\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (71,73,102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [164]:
df1.isnull().any(axis=1).sum()

8361

# Zoom in on a narrower set of features: Waterway adequacy (71), lanes on structure (28A), ADT (29), ADTT (109), deck structure type (107), deck protection (108C), deck condition rating (58)

In [165]:
new_df1 = df1[['STRUCTURE_NUMBER_008', 'WATERWAY_EVAL_071', 'TRAFFIC_LANES_ON_028A', 'ADT_029', 
               'DECK_COND_058', 'DECK_STRUCTURE_TYPE_107', 'DECK_PROTECTION_108C',
               'PERCENT_ADT_TRUCK_109']].copy()
new_df2 = df2[['STRUCTURE_NUMBER_008', 'WATERWAY_EVAL_071', 'TRAFFIC_LANES_ON_028A', 'ADT_029', 
               'DECK_COND_058', 'DECK_STRUCTURE_TYPE_107', 'DECK_PROTECTION_108C',
               'PERCENT_ADT_TRUCK_109']].copy()

In [166]:
new_df1.columns

Index(['STRUCTURE_NUMBER_008', 'WATERWAY_EVAL_071', 'TRAFFIC_LANES_ON_028A',
       'ADT_029', 'DECK_COND_058', 'DECK_STRUCTURE_TYPE_107',
       'DECK_PROTECTION_108C', 'PERCENT_ADT_TRUCK_109'],
      dtype='object')

In [167]:
len(new_df1)

8361

# Get unique bridge IDs from 1992 inspections

In [168]:
unique_bridge_ids_1992 = new_df1.STRUCTURE_NUMBER_008.unique().tolist()

In [169]:
unique_bridge_ids_1992

['003000000.0001A',
 '003000102.30010',
 '003000202.90011',
 '003000403.50012',
 '003000504.40013',
 '003000605.10014',
 '003001200.90007',
 '003001503.90006',
 '003010004.80008',
 '003010215.90009',
 '003010812.90002',
 '003011201.1005A',
 '003011202.00004',
 '003011203.7003A',
 '0335.01        ',
 '049000000.1030A',
 '049000101.1006A',
 '049000206.2004A',
 '049000300.40010',
 '049000304.90019',
 '049000400.10029',
 '049000609.00025',
 '049000800.20012',
 '049001001.10001',
 '049001100.1005A',
 '049001100.6007A',
 '049002000.1011A',
 '049002100.00013',
 '049002500.30003',
 '049003200.10021',
 '049003900.80009',
 '049005700.1018A',
 '049005700.70002',
 '049006400.10031',
 '049006601.00033',
 '049007300.9016A',
 '049008300.4023A',
 '049008400.70017',
 '049022800.90008',
 '049030200.40022',
 '049033000.50020',
 '049049101.5026A',
 '049062000.2024A',
 '049062701.40027',
 '049080400.20014',
 '0560.01        ',
 '0560.02        ',
 '0560.03        ',
 '057000500.2017A',
 '057000510.30015',


# Get unique bridge IDs from 2019

In [170]:
unique_bridge_ids_2019 = new_df2.STRUCTURE_NUMBER_008.unique().tolist()

# Find all common bridge IDs from 1992 and 2019 for comparison analysis

In [171]:
df1_as_set = set(unique_bridge_ids_1992)
df2_as_set = set(unique_bridge_ids_2019)
intersection_df1_df2 = df1_as_set.intersection(df2_as_set)
df1_minus_df2 = df1_as_set.difference(intersection_df1_df2)
df2_minus_df1= df2_as_set.difference(intersection_df1_df2)

In [172]:
len(intersection_df1_df2)

4891

In [173]:
len(df1_minus_df2)

2915

# Create lists from bridge IDs

In [174]:
list_of_ids_1992 = []
for index, row in new_df1.iterrows():
    list_of_ids_1992.append(row['STRUCTURE_NUMBER_008'])
    
list_of_ids_2019 = []
for index, row in new_df2.iterrows():
    list_of_ids_2019.append(row['STRUCTURE_NUMBER_008'])

In [175]:
len(list_of_ids_2019)

8785

# Remove whitespace from ids

In [176]:
new_list_of_ids_1992 = []
for bridge_id in list_of_ids_1992:
    temp = bridge_id.replace(' ', '')
    new_list_of_ids_1992.append(temp) 
    
new_list_of_ids_2019 = []
for bridge_id in list_of_ids_2019:
    temp = bridge_id.replace(' ', '')
    new_list_of_ids_2019.append(temp) 

# For all unique ids in common from 1992 and 2019, create two dictionaries of dataframe evaluation counts for each bridge id for each year. We observe that for the 1992 dataset the highest number of evaluations for a single bridge id is only 3. Similarly, for the 2019 dataset, the highest number of evaluations for a single bridge id is only 1.

In [177]:
import time
import progressbar

bar = progressbar.ProgressBar(maxval=len(new_list_of_ids_1992)).start()

dict_of_df1_bridge_id_counts = {}

for idx, bridge_id in enumerate(new_list_of_ids_1992):
    if bridge_id in dict_of_df1_bridge_id_counts.keys():
        dict_of_df1_bridge_id_counts[bridge_id] = dict_of_df1_bridge_id_counts[bridge_id] + 1
    else:
        dict_of_df1_bridge_id_counts[bridge_id] = 1
    bar.update(idx)

In [178]:
max(dict_of_df1_bridge_id_counts.values())

3

In [179]:
import time
import progressbar

bar = progressbar.ProgressBar(maxval=len(new_list_of_ids_2019)).start()

dict_of_df2_bridge_id_counts = {}

for idx, bridge_id in enumerate(new_list_of_ids_2019):
    if bridge_id in dict_of_df2_bridge_id_counts.keys():
        dict_of_df2_bridge_id_counts[bridge_id] = dict_of_df2_bridge_id_counts[bridge_id] + 1
    else:
        dict_of_df2_bridge_id_counts[bridge_id] = 1
    bar.update(idx)

In [180]:
max(dict_of_df2_bridge_id_counts.values())

1

# Based on our narrowed down set of features as well as examining only bridge ids that appear in both datasets, we now examine how many non-null rows exist in each year

In [181]:
new_df1_intersection = new_df1[new_df1.STRUCTURE_NUMBER_008.isin(intersection_df1_df2)]
new_df2_intersection = new_df2[new_df2.STRUCTURE_NUMBER_008.isin(intersection_df1_df2)]

In [182]:
len(new_df1_intersection)

5283

In [183]:
len(new_df2_intersection)

4891

# Based on the top five list of bridge IDs derived, compare non-null entries for each of the bridge IDs from both years based on the new narrower set of features

In [184]:
new_df1_intersection_non_null = new_df1_intersection.dropna()
print('Non-null entries for bridge IDs in the 1992 dataset that '
      'also occur in the 2019 dataset and for the narrower set of features is: ' 
      + str(len(new_df1_intersection_non_null)))

Non-null entries for bridge IDs in the 1992 dataset that also occur in the 2019 dataset and for the narrower set of features is: 4890


In [185]:
new_df2_intersection_non_null = new_df2_intersection.dropna()
print('Non-null entries for bridge IDs in the 1992 dataset that '
      'also occur in the 2019 dataset and for the narrower set of features is: ' 
      + str(len(new_df2_intersection_non_null)))

Non-null entries for bridge IDs in the 1992 dataset that also occur in the 2019 dataset and for the narrower set of features is: 4891
